<a href="https://colab.research.google.com/github/Shch2295404/Shch2295404/blob/main/Webinar_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оценка качества звонков

1. *.mp3 - записи звонков
2. Расшифровать звонок, и превратить его в текст
3. Мы отдаем текс в GPT, он делает оценку согластно чек листу
4. Сделать отчет

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata

OpenAI_api_key = userdata.get('OpenAI_api_key')
client = OpenAI(api_key=OpenAI_api_key)

In [ ]:
file_name = "/content/2024-06-25_[18_48_41]_112.mp3"

with open(file_name, "rb") as audio_file:
    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        response_format="text",
        prompt="Зерокодер, Зерокодинг, нейросети, от принципов к практике"
    )

output_file_name = file_name.replace(".mp3", ".txt")

with open(output_file_name, "w") as text_file:
    text_file.write(transcription)

print(transcription)

print(f"Transcription saved to {output_file_name}")

Ксения, здравствуйте, меня зовут Дмитрий, университет Зерокодер. Да, здрасте, слушаю. Ксения, вижу, ранее ставили заявочку на обучение на направление программирования на Python с арифм бизнес, все верно? Да-да-да, все верно, как раз. Замечательно, Ксения. Позвольте нам задать несколько вопросов по поводу вашей заявочки. Ксения, подскажите, а почему заинтересовались данным направлением? А, ну, я просто так на сторону вообще IT думаю. Да, я просто видела вот вашу рекламу, сходила на кабинар, вот, ну и как-то мне почетнее стала эта картина, вот я, не знаю, как-то уже хотела послушать, что вы предлагаете. Я вас понял. Ксения, подскажите, где планируете применять навыки программиста? Не расслышала ваш вопрос. Подскажите, пожалуйста, где планируете применять навыки программиста? А, ну, хотелось бы работать, зарабатывать в этом плане, то есть я в IT хочу. Вот. А, Ксения, подскажите, планируете полностью переквалифицироваться или подрабатывать на фрилансе? Ну, наверное, как-то и фриланс, ну, е

In [ ]:
with open("/content/prompt.txt", "r", encoding="utf-8") as file:
    system_prompt = file.read()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": transcription}
  ]
)

evaluation_result = completion.choices[0].message.content

print(evaluation_result)

Выявлена потребность:
    Оценка: да
    Комментарий: Менеджер успешно выявил потребность клиента в обучении программированию на Python и архитектуре бизнеса, учитывая, что клиент рассматривает возможность работы в IT сфере.

Консультация по продукту:
    Оценка: да
    Комментарий: Менеджер провел консультацию по продукту, объяснил структуру обучения, продвижение в профессии и доступ к материалам. Однако, не было конкретных деталей о содержании курса или учебных модулях.

Работа с возражениями:
    Оценка: нет
    Комментарий: Менеджер не смог адекватно отработать возражения клиента по поводу необходимости полной переквалификации или оформления расрочки. Повторные пояснения не проведены.

Мотивация на завершение сделки:
    Оценка: да
    Комментарий: Менеджер успешно мотивировал клиента на завершение сделки, предложив дополнительные опции и информацию о сроках и условиях оплаты. Клиент выразил готовность купить дополнительные треки по трудоустройству.

Повышение среднего чека:
    Оц

In [ ]:
import json

sections = evaluation_result.strip().split("\n\n")
parsed_data = {}

for section in sections:
    lines = section.split("\n")
    title = lines[0].strip(":")
    grade = lines[1].split(": ")[1].strip(",")
    comment = lines[2].split(": ", 1)[1]
    parsed_data[title] = {
        "Оценка": grade,
        "Комментарий": comment
    }

json_data = json.dumps(parsed_data, ensure_ascii=False, indent=4)
print(json_data)

{
    "Выявлена потребность": {
        "Оценка": "да",
        "Комментарий": "Менеджер успешно выявил потребность клиента в обучении программированию на Python и архитектуре бизнеса, учитывая, что клиент рассматривает возможность работы в IT сфере."
    },
    "Консультация по продукту": {
        "Оценка": "да",
        "Комментарий": "Менеджер провел консультацию по продукту, объяснил структуру обучения, продвижение в профессии и доступ к материалам. Однако, не было конкретных деталей о содержании курса или учебных модулях."
    },
    "Работа с возражениями": {
        "Оценка": "нет",
        "Комментарий": "Менеджер не смог адекватно отработать возражения клиента по поводу необходимости полной переквалификации или оформления расрочки. Повторные пояснения не проведены."
    },
    "Мотивация на завершение сделки": {
        "Оценка": "да",
        "Комментарий": "Менеджер успешно мотивировал клиента на завершение сделки, предложив дополнительные опции и информацию о сроках и усло

In [ ]:
import json
import csv

data = json.loads(json_data)

csv_file_name = file_name.replace(".mp3", ".csv")

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Запись заголовков
    writer.writerow(["Этап", "Оценка", "Комментарии"])

    # Запись данных из JSON
    for key, value in data.items():
        writer.writerow([key, value["Оценка"], value["Комментарий"]])

print(f"Данные успешно сохранены в файл {csv_file_name}")

Данные успешно сохранены в файл /content/2024-06-25_[18_48_41]_112.csv


In [ ]:
import requests
import time
from google.colab import userdata

def send_file(api_key, file_path, language, speakers):
    url = f'https://speech2text.ru/api/recognitions/task/file?api-key={api_key}'
    with open(file_path, 'rb') as file:
        files = {'file': file}
        params = {'lang': language, 'speakers': speakers}
        print(f"Отправка файла {file_path} на распознавание...")
        response = requests.post(url, files=files, params=params)
        data = response.json()
        task_id = data.get('id')
        print(f"Получен ID задачи: {task_id}")
        return task_id

def check_status(api_key, task_id):
    url = f'https://speech2text.ru/api/recognitions/{task_id}?api-key={api_key}'
    print(f"Проверка статуса для ID задачи: {task_id}")
    start_time = time.time()

    while True:
        response = requests.get(url)
        data = response.json()
        status = data.get('status', {}).get('code')
        elapsed_time = time.time() - start_time
        print(f"Прошло времени: {int(elapsed_time)} секунд. Файл распознается")
        if status == 200:
            print("\nЗадача успешно завершена.")
            break
        time.sleep(10)

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Время, затраченное на транскрибацию: {total_time:.2f} секунд")

def get_result(api_key, task_id, format):
    url = f'https://speech2text.ru/api/recognitions/{task_id}/result/{format}?api-key={api_key}'
    response = requests.get(url)
    result = response.content.decode('utf-8')  # Декодируем байты в строку
    return result

if __name__ == "__main__":
    api_key = userdata.get('api_key')
    file_path = file_name
    language = 'ru'
    speakers = 2
    format = 'txt'
    task_id = send_file(api_key, file_path, language, speakers)
    check_status(api_key, task_id)
    s2t_transcription = get_result(api_key, task_id, format)
    print(s2t_transcription)

    output_file_name = file_name.replace(".mp3", "s2t.txt")

    with open(output_file_name, "w") as text_file:
      text_file.write(s2t_transcription)

    print(f"Transcription saved to {output_file_name}")

Отправка файла /content/2024-06-25_[18_48_41]_112.mp3 на распознавание...
Получен ID задачи: LK1xY768VWLvop0gdRBwixZJHH1IntNc
Проверка статуса для ID задачи: LK1xY768VWLvop0gdRBwixZJHH1IntNc
Прошло времени: 0 секунд. Файл распознается
Прошло времени: 12 секунд. Файл распознается
Прошло времени: 23 секунд. Файл распознается
Прошло времени: 34 секунд. Файл распознается
Прошло времени: 45 секунд. Файл распознается
Прошло времени: 56 секунд. Файл распознается
Прошло времени: 68 секунд. Файл распознается
Прошло времени: 79 секунд. Файл распознается
Прошло времени: 90 секунд. Файл распознается
Прошло времени: 101 секунд. Файл распознается
Прошло времени: 112 секунд. Файл распознается
Прошло времени: 123 секунд. Файл распознается
Прошло времени: 134 секунд. Файл распознается
Прошло времени: 145 секунд. Файл распознается
Прошло времени: 156 секунд. Файл распознается

Задача успешно завершена.
Время, затраченное на транскрибацию: 156.78 секунд

Спикер 1:
0:00:23 - Ксения, здравствуйте, меня зов

In [ ]:
with open("/content/prompt.txt", "r", encoding="utf-8") as file:
    system_prompt = file.read()

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": s2t_transcription}
  ]
)

evaluation_result = completion.choices[0].message.content

print(evaluation_result)


Выявлена потребность:
    Оценка: да,
    Комментарий: Менеджер выявил основные потребности клиента: желание получить навыки программирования, чтобы использовать их в будущем на фрилансе или при полной переквалификации (вопросы на 0:01:23, 0:01:44, 0:02:01).

Консультация по продукту:
    Оценка: да,
    Комментарий: Менеджер подробно рассказал о курсе, его содержании и о дополнительных модулях, например, про возможность выбора между карьерой и фрилансом, а также про дополнительные опции (начиная с 0:03:48 и далее).

Работа с возражениями:
    Оценка: да,
    Комментарий: Менеджер несколько раз переспрашивал и уточнил непонятные для клиента моменты, например, про дополнительные опции и возможность продления доступа, а также работу с рассрочкой через банки (вопросы на 0:06:22, 0:08:07, 0:10:21).

Мотивация на завершение сделки:
    Оценка: да,
    Комментарий: Менеджер замотивировал клиента на покупку дополнительных модулей и продвижение заявки на рассрочку (вопросы на 0:08:50, 0:09:02)

In [ ]:
import json

sections = evaluation_result.strip().split("\n\n")
parsed_data = {}

for section in sections:
    lines = section.split("\n")
    title = lines[0].strip(":")
    grade = lines[1].split(": ")[1].strip(",")
    comment = lines[2].split(": ", 1)[1]
    parsed_data[title] = {
        "Оценка": grade,
        "Комментарий": comment
    }

json_data = json.dumps(parsed_data, ensure_ascii=False, indent=4)
print(json_data)

{
    "Выявлена потребность": {
        "Оценка": "да",
        "Комментарий": "Менеджер выявил основные потребности клиента: желание получить навыки программирования, чтобы использовать их в будущем на фрилансе или при полной переквалификации (вопросы на 0:01:23, 0:01:44, 0:02:01)."
    },
    "Консультация по продукту": {
        "Оценка": "да",
        "Комментарий": "Менеджер подробно рассказал о курсе, его содержании и о дополнительных модулях, например, про возможность выбора между карьерой и фрилансом, а также про дополнительные опции (начиная с 0:03:48 и далее)."
    },
    "Работа с возражениями": {
        "Оценка": "да",
        "Комментарий": "Менеджер несколько раз переспрашивал и уточнил непонятные для клиента моменты, например, про дополнительные опции и возможность продления доступа, а также работу с рассрочкой через банки (вопросы на 0:06:22, 0:08:07, 0:10:21)."
    },
    "Мотивация на завершение сделки": {
        "Оценка": "да",
        "Комментарий": "Менеджер замо

In [ ]:
import json
import csv

data = json.loads(json_data)

csv_file_name = file_name.replace(".mp3", "s2t.csv")

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Запись заголовков
    writer.writerow(["Этап", "Оценка", "Комментарии"])

    # Запись данных из JSON
    for key, value in data.items():
        writer.writerow([key, value["Оценка"], value["Комментарий"]])

print(f"Данные успешно сохранены в файл {csv_file_name}")

Данные успешно сохранены в файл /content/2024-06-25_[18_48_41]_112s2t.csv
